In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

input_dims = (1,120,1,)
l1_output_dim = 84
l2_output_dim = 10

t_max = 2**9

np.random.seed(seed=48)
weights1 = (np.random.rand(l1_output_dim, np.product(input_dims)) - 0.5) / 2
biases1 = (np.random.rand(l1_output_dim) - 0.5) / 2
weights2 = (np.random.rand(l2_output_dim, l1_output_dim) - 0.5) / 2
biases2 = (np.random.rand(l2_output_dim) - 0.5) / 2

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.Dense(weights=weights1, biases=biases1),
    layers.Dense(weights=weights2, biases=biases2),
    layers.MonitorLayer(),
])

hidden7 = quartz.probe(loihi_model.layers[2].blocks[18])
calc7 = quartz.probe(loihi_model.layers[2].blocks[18].neurons[0])

values = np.random.rand(np.product(input_dims)) / 2
inputs = quartz.utils.decode_values_into_spike_input(values, t_max)
quantized_values = (values*t_max).round()/t_max
quantized_weights1 = (weight_acc*weights1).round()/weight_acc
quantized_weights2 = (weight_acc*weights2).round()/weight_acc
quantized_biases1 = (biases1*t_max).round()/t_max
quantized_biases2 = (biases2*t_max).round()/t_max

model = nn.Sequential(
    nn.Linear(in_features=np.product(input_dims), out_features=l1_output_dim), nn.ReLU(),
    nn.Linear(in_features=l1_output_dim, out_features=l2_output_dim), nn.ReLU(),
)
model[0].weight = nn.Parameter(torch.tensor(quantized_weights1))
model[0].bias = nn.Parameter(torch.tensor(quantized_biases1))
model[2].weight = nn.Parameter(torch.tensor(quantized_weights2))
model[2].bias = nn.Parameter(torch.tensor(quantized_biases2))
model_output = model(torch.tensor(quantized_values)).detach().numpy()
model_output

In [ ]:
loihi_model(inputs, t_max)

In [ ]:
loihi_model.layers[0].weight_e

In [ ]:
hidden7.output()

In [ ]:
calc7.plot()

In [ ]:
hidden7.plot()

## Conv Maxpool layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

input_dims = ( 1,2,2)
weight_dims = ( 3,1,2,2)

# (( 1,32,32), ( 6,1,5,5), 251),
# (( 6,14,14), (8,6,5,5), 251),
t_max = 2**9
weight_acc = 128
conv_kernel_size = weight_dims[2:]
pooling_kernel_size = [2,2]
pooling_stride = 2

np.random.seed(seed=47)
weights = (np.random.rand(*weight_dims)-0.5) / 4
biases = (np.random.rand(weight_dims[0])-0.5) / 2

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.Conv2D(weights=weights, biases=biases),
    layers.MaxPool2D(kernel_size=pooling_kernel_size, stride=pooling_stride),
    layers.MonitorLayer(),
])

trigg0 = quartz.probe(loihi_model.layers[1].blocks[0])
conv00 = quartz.probe(loihi_model.layers[1].blocks[1].neurons[0])
conv01 = quartz.probe(loihi_model.layers[1].blocks[1].neurons[1])
conv20 = quartz.probe(loihi_model.layers[1].blocks[7].neurons[0])
conv21 = quartz.probe(loihi_model.layers[1].blocks[7].neurons[1])
pool0 = quartz.probe(loihi_model.layers[2].blocks[0])
pool1 = quartz.probe(loihi_model.layers[2].blocks[0])

values = np.random.rand(np.product(input_dims))
inputs = quartz.utils.decode_values_into_spike_input(values, t_max)
quantized_values = (values*t_max).round()/t_max
quantized_weights = (weight_acc*weights).round()/weight_acc
quantized_biases = (biases*t_max).round()/t_max

model = nn.Sequential(
    nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=conv_kernel_size), nn.ReLU(),
    nn.MaxPool2d(kernel_size=pooling_kernel_size, stride=pooling_stride), nn.ReLU(),
)
model[0].weight = nn.Parameter(torch.tensor(quantized_weights))
model[0].bias = nn.Parameter(torch.tensor(quantized_biases))
model_output = model(torch.tensor(quantized_values.reshape(1, *input_dims[:3]))).squeeze().detach().numpy()
model_output.flatten()

In [ ]:
loihi_model(inputs, t_max)

In [ ]:
biases

In [ ]:
trigg0.plot()

In [ ]:
conv20.plot()

In [ ]:
conv21.plot()

In [ ]:
pool0.plot()


In [ ]:
loihi_model.layers[1].blocks[1].neurons[0].incoming_synapses()

In [ ]:
conv00.plot()

In [ ]:
conv01.plot()